In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
from ast import literal_eval
import pandas as pd
import random
import torch
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

# TODO: add your openai api key here
openai.organization =
openai.api_key = 

# colab
DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cpu')

## Load data

In [ ]:
infoSheet = pd.read_csv(DATA_PATH + 'data/openai_embedding/infoSheet_name_abstract.csv')
# # Drop rows where 'taxonomy heading ids' is '0'
# infoSheet.drop(infoSheet[infoSheet['taxonomy heading ids'] == '0'].index, inplace = True)
# # Drop rows where 'abstract_en' is NaN
# infoSheet.dropna(subset=['abstract_en'], inplace=True)
# infoSheet.reset_index(drop=True, inplace=True)
infoSheet['embedding'] = infoSheet['embedding'].apply(literal_eval)
print(infoSheet.isnull().sum())
print('Length: ', len(infoSheet.index))
infoSheet.head(10)

ID                       0
name_en                  0
name_fr                 87
abstract_en              0
abstract_fr             89
description_en           0
description_fr          87
taxonomy heading ids     0
embedding                0
dtype: int64
Length:  230


,ID,name_en,name_fr,abstract_en,abstract_fr,description_en,description_fr,taxonomy heading ids,embedding
0,8920,Abuse and Domestic Violence,Maltraitance et violence familiale,"Abuse is behaviour used to intimidate, isolate...",La maltraitance est un comportement visant à i...,\r\n\tWhat is Abuse and Domestic Violence?\r\n...,\r\n\tQu&#39;est-ce que la maltraitance et la ...,"0,219,58,8,7,6,50,9,36,54,37","[-0.006836807355284691, -1.2969143426744267e-0..."
1,69660,"ADHD in Children, Youth and Adults: Informatio...",NaN,Attention deficit hyperactivity disorder (ADHD...,NaN,"\r\n\tAbbreviations\r\n\r\n\tADHD, attention-d...",NaN,"0,13","[6.297678191913292e-05, 0.005104047246277332, ..."
2,20505,ADHD: Managing ADHD Medications,TDAH: Gestion des médicaments pour le TDAH,Although ADHD medications may be helpful for s...,Bien que les médicaments du TDAH peuvent être ...,\r\n\tGeneral Considerations\r\n\r\n\t\r\n\t\t...,\r\n\tObservations générales\r\n\r\n\t\r\n\t\t...,"0,13","[-0.007076850160956383, -0.0020307896193116903..."
3,8917,Alcohol and Substance Use Problems in Children...,Problèmes de l’alcoolisme et la toxicomanie ch...,Many children/youth experiment with alcohol an...,Nombreux sont les enfants et les jeunes qui fo...,\r\n\tDavid's Story\r\n\r\n\tUp until this sch...,\r\n\tL’histoire de David\r\n\r\n\tJusqu’à cet...,"0,12","[-0.0038117102812975645, -0.008029664866626263..."
4,8909,Alcohol: Cutting Back or Quitting Drinking,Problèmes liés à la consommation d'alcool,Drinking alcohol is an accepted practice in ma...,La consommation d'alcool est une pratique acce...,\r\n\tIntroduction\r\n\r\n\tMany people who dr...,\r\n\tIntroduction\r\n\r\n\tNombreuses sont le...,"0,12","[-0.010912169702351093, 0.0006169816479086876,..."
5,61003,Alcohol Use Disorder in Adults: Information fo...,NaN,Alcohol use problems are common in primary car...,NaN,\r\n\tEpidemiology\r\n\r\n\tPrevalence\r\n\r\n...,NaN,"0,12,266","[-0.0002503542636986822, 0.0016840933822095394..."
6,18393,Alcohol Use Disorder in Youth: Information for...,NaN,Many youth will experiment with alcohol and su...,NaN,Epidemiology\r\n\r\n\t\r\n\t\t10% of the&nbsp;...,NaN,"0,12,266","[-0.007133564446121454, -0.0029860695358365774..."
7,24390,Alzheimer's Dementia: Information for Primary ...,NaN,Alzheimer’s is the most common form of dementi...,NaN,\r\n\tCase\r\n\r\n\t\r\n\t\tYour 80 year old p...,NaN,"0,79","[-0.0032352525740861893, 0.0039140209555625916..."
8,11564,Anger,La colère,Anger is a normal emotion that we get when we ...,La colère est une émotion normale que nous res...,\r\n\tDoes any of this sound like you?\r\n\r\n...,\r\n\tEst-ce que cela trouve un &eacute;cho ch...,"0,14,50,54,9,36,37","[-0.0030424355063587427, 0.006882417947053909,..."
9,26975,Anorexia Nervosa: Information for Primary Care,NaN,Anorexia nervosa is a common eating disorder a...,NaN,\r\n\tCase\r\nIdentifying data&nbsp;32-yo fema...,NaN,"0,22","[-3.332154960844491e-07, -0.000195899716345593..."


In [ ]:
taxonomy = pd.read_csv(DATA_PATH + 'data/openai_embedding/taxonomy_name_description.csv')
taxonomy['embedding'] = taxonomy['embedding'].apply(literal_eval)
taxonomy.dropna(subset=['description'], inplace=True)
taxonomy.reset_index(drop=True, inplace=True)
print(taxonomy.isnull().sum())
print('Length: ', len(taxonomy.index))
taxonomy.head(10)

id              0
name            0
description     0
translations    0
embedding       0
dtype: int64
Length:  192


,id,name,description,translations,embedding
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":...","[-0.01242552325129509, 0.0049454341642558575, ..."
1,2,All Mental Health Resources,The listings of mental health resources includ...,"{""name"":{""en"":""All Mental Health Resources"",""f...","[-0.003916683606803417, 0.0043426863849163055,..."
2,3,Crisis and Emergency,Refers to all programs that provide immediate ...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res...","[-0.0016540151555091143, 0.004231865983456373,..."
3,4,"System Navigation, including Information and R...","Are you looking for help, but dont know where ...","{""name"":{""en"":""System Navigation, including In...","[-0.0052290805615484715, 0.001998607302084565,..."
4,5,Child Welfare including Children's Aid Society...,The child welfare / child protection system is...,"{""name"":{""en"":""Child Welfare including Childre...","[-0.0017937872326001525, 0.002536701038479805,..."
5,6,Emergency Shelter and Housing,There are various shelters that people can use...,"{""name"":{""en"":""Emergency Shelter and Housing"",...","[-0.003927853424102068, 0.005480578169226646, ..."
6,7,Hospital Emergency Department,Is there an emergency such as medical or menta...,"{""name"":{""en"":""Hospital Emergency Department"",...","[-0.01046136673539877, 0.001888505183160305, -..."
7,8,"Crisis Lines including Telephone, Online and Chat","Are you in a crisis? Crisis lines offer free, ...","{""name"":{""en"":""Crisis Lines including Telephon...","[-0.006740534212440252, -0.004541291855275631,..."
8,9,Psychiatrists,Psychiatrists are medical doctors who have rec...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre...","[0.00967669952660799, 0.0008635374833829701, -..."
9,10,A-Z Mental Health Conditions and Topics,Alphabetical list of mental health topics and ...,"{""name"":{""en"":""A-Z Mental Health Conditions an...","[-0.010088824667036533, 0.0018147194059565663,..."


## Log Probabilities

In [ ]:
# response = openai.Completion.create(
#   engine='text-davinci-003',  # Choose the engine that suits your needs
#   prompt=prompt,
#   max_tokens=10,
#   logprobs=2,
#   temperature=0
# )
# response

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a psychiatrist. You need to select an article that is \
              most relevant to the user's problem among several articles provided by the user according \
              to the user's problem. You don't need to explain why you chose."},
    {"role": "user", "content": "Question: Brainspotting. Article 1: Learning Disorders. Article 2: Asperger Syndrome. Article 3: Behaviour and Conduct Problems."}
  ]
)

print(completion.choices[0].message['content'])

Article 1: Learning Disorders.


## Cosine Simlirity

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices = [113, 10, 124, 39, 88, 23, 69, 13, 179, 218]

In [ ]:
def search(queries, docs, query_indices, topK=10):
    if topK > len(docs):
        topK = len(docs)

    result = []
    for idx, query in queries.iterrows():
        if idx in query_indices:
            tmp = []
            query_embedding = query['embedding']
            for _, doc in docs.iterrows():
                doc_embedding = doc['embedding']
                tmp.append(cosine_similarity(doc_embedding, query_embedding))
            top_results = torch.topk(torch.tensor(tmp), topK)
            result.append(top_results)

    return result

In [ ]:
result = search(infoSheet, taxonomy, search_term_indices)

predictions = {'infoSheet_index': [], 'infoSheet_name': [], 'pred_taxonomy_index': [], 'pred_taxonomy_name': [], 'score': [], 'gold_taxonomy_index': []}
for q in range(len(result)):
    for score, idx in zip(result[q][0], result[q][1]):
        predictions['infoSheet_index'].append(search_term_indices[q])
        predictions['infoSheet_name'].append(infoSheet['name_en'][search_term_indices[q]])
        predictions['pred_taxonomy_index'].append(idx.item()+1)
        predictions['pred_taxonomy_name'].append(taxonomy['name'][idx.item()])
        predictions['score'].append(score.item())
        predictions['gold_taxonomy_index'].append(infoSheet['taxonomy heading ids'][search_term_indices[q]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

,infoSheet_index,infoSheet_name,pred_taxonomy_index,pred_taxonomy_name,score,gold_taxonomy_index
0,113,Learning Disabilities,141,"Antisocial Personality Disorder (ASPD), Psycho...",0.795644,"[0, 25, 98, 220, 36]"
1,113,Learning Disabilities,136,Multisystemic Therapy (MST),0.755436,"[0, 25, 98, 220, 36]"
2,113,Learning Disabilities,27,Schizophrenia and Psychosis,0.755396,"[0, 25, 98, 220, 36]"
3,113,Learning Disabilities,23,Fetal Alcohol and Fetal Alcohol Spectrum Disor...,0.752822,"[0, 25, 98, 220, 36]"
4,113,Learning Disabilities,49,"Family, Marriage and Couple Therapists",0.750760,"[0, 25, 98, 220, 36]"
5,113,Learning Disabilities,102,Borderline Personality Disorder (BPD),0.749998,"[0, 25, 98, 220, 36]"
6,113,Learning Disabilities,67,Tourette Syndrome and Tic Disorders,0.749825,"[0, 25, 98, 220, 36]"
7,113,Learning Disabilities,24,"Family, Marital, and Couple Therapy",0.749064,"[0, 25, 98, 220, 36]"
8,113,Learning Disabilities,15,Autism and Autism Spectrum Disorders,0.748723,"[0, 25, 98, 220, 36]"
9,113,Learning Disabilities,50,Family Physicians,0.747422,"[0, 25, 98, 220, 36]"


In [ ]:
predictions.to_csv(DATA_PATH + 'data/predictions/openai-cosine.csv', index=False)